In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from multiprocessing.pool import ThreadPool
from multiprocessing import cpu_count
max_cpu = cpu_count()

In [98]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

#### Part (a)



#### Part (b)



#### Part (c)



#### Part (d)



In [99]:
def DGP():
    Z = np.random.uniform(0,1,size=(500,10))
    epsilon, nu = np.random.normal(0,0.25,size=(500,1)), np.random.normal(0,0.25,size=(500,1))
    g_z = np.multiply(Z[:,1],np.add(Z[:,2],Z[:,3]))
    X = g_z.reshape(500,1) + nu
    m_xz = np.multiply(X,(np.add(Z[:,1],Z[:,2]) ** 2).reshape(500,1))
    y = m_xz.reshape(500,1) + epsilon

    gamma = np.random.normal(size=(10,1))
    p_z = np.exp(-Z @ gamma) / (1 + np.exp(-Z @ gamma))
    zeta = np.random.uniform(0,1)
    D = np.where((p_z - zeta) > 0,1,0).reshape(500,1)
    
    return np.concatenate((y,X,Z,D),axis=1)

def cv(data,k=5):
    n = X.shape[0]
    I_sections = np.array_split(data,5)
    # # creating indices for folds
    # folds = []
    # remaining_idx = np.arange(n)
    # for i in np.arange(k):
    #     indices = np.random.choice(remaining_idx,round(n * 1/k), replace=False)
    #     remaining_idx = np.setdiff1d(remaining_idx, indices)
    #     folds.append(indices)
    
    # 
    thetas = []
    for i,_ in enumerate(I_sections):
        I_k = I_sections[i]
        I_minusk_sections = I_sections[:i] + I_sections[i+1:]
        I_minusk = np.concatenate(I_minusk,axis=0)
        
        # estimating theta_k
        ## estimating nuisance parameters
        Z = I_minusk[:,3:13]
        data_D1 = I_minusk[I_minusk[:,-1] == 1]
        Z_D1 = I_minusk[:,3:13]
        
        phi_rf = RandomForestRegressor(random_state=14)
        phi_cv = GridSearchCV(phi_rf, n_jobs = -1,
                              param_grid = {'max_features':np.arange(1,11)})
        phi_cv.fit(Z_D1,I_minusk[:,0] * I_minusk[:,1])
        
        pi_rf = RandomForestRegressor(random_state=14)
        pi_cv = GridSearchCV(pi_rf, n_jobs = -1,
                             param_grid = {'max_features':np.arange(1,11)})
        pi_cv.fit(Z_D1,I_minusk[:,1] ** 2)
        
        prop_rf = RandomForestRegressor(random_state=14)
        prop_cv = GridSearchCV(prop_rf, n_jobs = -1,
                               param_grid = {'max_features':np.arange(1,11)})
        prop_cv.fit(Z,I_minusk[:,-1])
        
        phi_hat = phi_cv.predict(I_k)
        pi_hat = pi_cv.predict(I_k)
        prop_hat = prop_cv.predict(I_k)
        
        ## calculate theta_k
        
        
        
        # variance estimator
        ## 
        
        
        X_train, X_test, Y_train, Y_test = train_test_split(X,Y,folds,i)
        betas = lasso(X_train,Y_train,lambda_)
        predicted = X_test @ betas
        error = sse(predicted,Y_test)
        total_error += error
    return thetas

In [100]:
data = DGP()

In [101]:
Z = data[:,3:13]

In [102]:
data_D1 = data[data[:,-1] == 1]
Z_D1 = data_D1[:,3:13]

In [117]:
I_sections = np.array_split(data,5)
I_k = I_sections[i]
I_minusk = I_sections[:i] + I_sections[i+1:]

In [119]:
np.concatenate(I_minusk,axis=0).shape

(400, 13)